In [ ]:
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import pandas_datareader.data as web
from tabulate import tabulate # Para tablas más estéticas en la consola
from colorama import Fore, Style, init # Para colores en la consola

# Inicializar colorama para que los colores funcionen en la consola y se reseteen automáticamente
init(autoreset=True)

# --- Funciones de Cálculo de Indicadores Técnicos ---

def calculate_ema(data_series, window):
    """Calcula la Media Móvil Exponencial (EMA) para una serie de datos."""
    if data_series.empty:
        return pd.Series(np.nan, index=data_series.index)
    return data_series.ewm(span=window, adjust=False).mean()

def calculate_rsi(data_series, window=14):
    """Calcula el Índice de Fuerza Relativa (RSI)."""
    if data_series.empty or len(data_series) < window:
        return pd.Series(np.nan, index=data_series.index)
    
    delta = data_series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_atr(data, window=14):
    """Calcula el Average True Range (ATR)."""
    if data.empty or len(data) < window:
        return pd.Series(np.nan, index=data.index)

    high_low = data['high'] - data['low']
    high_prev_close = abs(data['high'] - data['close'].shift())
    low_prev_close = abs(data['low'] - data['close'].shift())

    true_range = pd.DataFrame({'high_low': high_low, 'high_prev_close': high_prev_close, 'low_prev_close': low_prev_close}).max(axis=1)
    atr = true_range.ewm(span=window, adjust=False).mean()
    return atr

def calculate_macd(data_series, fast_period=12, slow_period=26, signal_period=9):
    """Calcula el Moving Average Convergence Divergence (MACD)."""
    if data_series.empty or len(data_series) < slow_period:
        return pd.DataFrame(index=data_series.index, columns=['macd', 'signal_line', 'macd_histogram'])

    exp1 = data_series.ewm(span=fast_period, adjust=False).mean()
    exp2 = data_series.ewm(span=slow_period, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal_period, adjust=False).mean()
    macd_histogram = macd - signal_line
    
    macd_df = pd.DataFrame({'macd': macd, 'signal_line': signal_line, 'macd_histogram': macd_histogram})
    return macd_df

def calculate_vwap_with_bands(data, std_period=20, std_multiplier=1.0):
    """
    Calcula el Volumen Ponderado de Precio Promedio (VWAP) y sus bandas de desviación estándar.

    El VWAP es el "precio justo" ajustado por volumen. Las bandas dan contexto de volatilidad
    y posibles zonas de sobrecompra/sobreventa respecto al VWAP.

    Fórmula VWAP: Sum(Precio Típico * Volumen) / Sum(Volumen)
    Precio Típico = (High + Low + Close) / 3
    Bandas: VWAP +/- (Multiplicador * Desviación Estándar del Precio Típico respecto al VWAP)

    Args:
        data (pd.DataFrame): DataFrame con columnas 'high', 'low', 'close', 'Volume'.
        std_period (int): Período para calcular la desviación estándar de las bandas.
        std_multiplier (float): Multiplicador para la desviación estándar de las bandas.

    Returns:
        pd.DataFrame: DataFrame original con columnas 'vwap', 'vwap_upper_band', 'vwap_lower_band'.
    """
    # Asegúrate de que las columnas necesarias estén presentes y tengan datos
    required_cols = ['high', 'low', 'close', 'Volume']
    if not all(col in data.columns and data[col].notna().any() for col in required_cols):
        print(f"{Fore.RED}Error: Faltan columnas o datos válidos para calcular VWAP. Se requieren: {required_cols}. Columnas disponibles: {data.columns.tolist()}{Style.RESET_ALL}")
        data['vwap'] = np.nan
        data['vwap_upper_band'] = np.nan
        data['vwap_lower_band'] = np.nan
        return data

    # Calcular el Precio Típico
    typical_price = (data['high'] + data['low'] + data['close']) / 3

    # Asegurarse de que el índice es de tipo datetime para operaciones acumulativas
    if not isinstance(data.index, pd.DatetimeIndex):
        data.index = pd.to_datetime(data.index)

    # Calcular el numerador y denominador del VWAP de forma acumulativa
    price_volume = typical_price * data['Volume']
    cumulative_pv = price_volume.cumsum() # Suma acumulada de (Precio Típico * Volumen)
    cumulative_volume = data['Volume'].cumsum() # Suma acumulada de Volumen

    # Evitar división por cero si el volumen acumulado es cero
    vwap = cumulative_pv / cumulative_volume.replace(0, np.nan)
    data['vwap'] = vwap

    # Calcular la Desviación Estándar del Precio Típico respecto al VWAP
    # Se utiliza una ventana móvil para la desviación estándar para que sea más dinámica
    vwap_std = (typical_price - vwap).rolling(window=std_period).std()
    
    data['vwap_upper_band'] = vwap + (vwap_std * std_multiplier)
    data['vwap_lower_band'] = vwap - (vwap_std * std_multiplier)
    
    return data

def calculate_anchored_vwap(data, anchor_date_str):
    """
    Calcula el Anchored VWAP (AVWAP) desde una fecha específica.
    
    Args:
        data (pd.DataFrame): DataFrame con columnas 'high', 'low', 'close', 'Volume'.
        anchor_date_str (str): Fecha de anclaje (YYYY-MM-DD).
        
    Returns:
        pd.Series: Serie del Anchored VWAP.
    """
    anchor_date = pd.to_datetime(anchor_date_str)
    
    # Filtrar datos desde la fecha de anclaje
    anchored_data = data[data.index >= anchor_date].copy()

    if anchored_data.empty:
        return pd.Series(np.nan, index=data.index)
        
    typical_price = (anchored_data['high'] + anchored_data['low'] + anchored_data['close']) / 3
    price_volume = typical_price * anchored_data['Volume']
    
    cumulative_pv = price_volume.cumsum()
    cumulative_volume = anchored_data['Volume'].cumsum()
    
    avwap = cumulative_pv / cumulative_volume.replace(0, np.nan)
    
    # Combinar AVWAP con el DataFrame original, rellenando NaN antes de la fecha de anclaje
    full_avwap = pd.Series(np.nan, index=data.index)
    full_avwap.update(avwap)
    return full_avwap

# --- Obtención de Datos Históricos Reales (yfinance y pandas_datareader) ---

def fetch_historical_data_real(ticker, start_date_str, end_date_for_api_str):
    """
    Intenta obtener datos históricos reales usando yfinance y luego pandas_datareader (Stooq) como fallback.
    
    Args:
        ticker (str): Símbolo del activo.
        start_date_str (str): Fecha de inicio para la descarga (YYYY-MM-DD).
        end_date_for_api_str (str): Fecha de fin para la descarga (YYYY-MM-DD), exclusiva en yfinance.
                                    Debe ser (hoy + 1 día) si se quiere incluir hoy.
    Returns:
        pd.DataFrame: DataFrame con datos históricos o vacío si la descarga falla desde ambas fuentes.
    """
    df = pd.DataFrame()
    
    print(f"\n{Fore.CYAN}Intentando descargar datos REALES para {ticker} desde {start_date_str} hasta {end_date_for_api_str} (exclusivo para APIs)...{Style.RESET_ALL}")

    # --- Intento 1: Usar yfinance ---
    print(f"{Fore.BLUE}  > Intento 1/2: Con yfinance...{Style.RESET_ALL}")
    yf_attempts = [
        {'method': 'download', 'params': {'start': start_date_str, 'end': end_date_for_api_str, 'interval': '1d', 'auto_adjust': True, 'actions': False, 'progress': False, 'show_errors': False}},
        {'method': 'ticker_history', 'params': {'start': start_date_str, 'end': end_date_for_api_str, 'interval': '1d', 'auto_adjust': True, 'actions': False}},
        {'method': 'download', 'params': {'period': 'max', 'interval': '1d', 'auto_adjust': True, 'actions': False, 'progress': False, 'show_errors': False}},
        {'method': 'ticker_history', 'params': {'period': 'max', 'interval': '1d', 'auto_adjust': True, 'actions': False}}
    ]
    
    for i, attempt in enumerate(yf_attempts):
        try:
            if attempt['method'] == 'download':
                df = yf.download(ticker, **attempt['params'])
            elif attempt['method'] == 'ticker_history':
                ticker_obj = yf.Ticker(ticker)
                history_params = {k: v for k, v in attempt['params'].items() if k not in ['progress', 'show_errors']}
                df = ticker_obj.history(**history_params)

            if not df.empty:
                # Filtrar si se usó 'period' para asegurar que el rango sea el deseado
                if 'period' in attempt['params']:
                    df = df[(df.index >= start_date_str) & (df.index < end_date_for_api_str)] 
                
                if not df.empty:
                    df.columns = [col.lower() for col in df.columns]
                    if 'volume' in df.columns:
                        df['volume'] = pd.to_numeric(df['volume'], errors='coerce')
                        df.rename(columns={'volume': 'Volume'}, inplace=True)
                    else:
                        print(f"{Fore.YELLOW}Advertencia: Columna 'volume' no encontrada en datos de yfinance. Necesaria para VWAP.{Style.RESET_ALL}")
                        return pd.DataFrame() 
                    print(f"{Fore.GREEN}  > ¡Éxito con yfinance!{Style.RESET_ALL}")
                    return df
        except Exception as e:
            pass 

    print(f"{Fore.YELLOW}  > yfinance no pudo obtener datos. Pasando a la siguiente fuente.{Style.RESET_ALL}")
    
    # --- Intento 2: Usar pandas_datareader con Stooq ---
    print(f"{Fore.BLUE}  > Intento 2/2: Con pandas_datareader (Stooq)...{Style.RESET_ALL}")
    try:
        df_stooq = web.DataReader(ticker, 'stooq', start=start_date_str, end=end_date_for_api_str)
        
        if not df_stooq.empty:
            df_stooq.columns = [col.lower() for col in df_stooq.columns]
            if 'volume' in df_stooq.columns:
                df_stooq['volume'] = pd.to_numeric(df_stooq['volume'], errors='coerce')
                df_stooq.rename(columns={'volume': 'Volume'}, inplace=True)
            else:
                print(f"{Fore.YELLOW}Advertencia: Columna 'volume' no encontrada en datos de Stooq. Necesaria para VWAP.{Style.RESET_ALL}")
                return pd.DataFrame() 
            
            df_stooq = df_stooq.sort_index()
            df_stooq = df_stooq[(df_stooq.index >= start_date_str) & (df_stooq.index < end_date_for_api_str)]
            
            if not df_stooq.empty:
                print(f"{Fore.GREEN}  > ¡Éxito con Stooq (pandas_datareader)!{Style.RESET_ALL}")
                return df_stooq
        print(f"{Fore.YELLOW}  > pandas_datareader (Stooq) no pudo obtener datos.{Style.RESET_ALL}")
    except Exception as e:
        print(f"{Fore.RED}  > Error en pandas_datareader (Stooq): {e}{Style.RESET_ALL}")
        
    print(f"\n{Fore.RED}¡Error Crítico! No se pudieron descargar datos REALES de ninguna fuente fiable.{Style.RESET_ALL}")
    return pd.DataFrame()

# --- Generación de Datos Históricos Simulados (FALLBACK) ---

def fetch_simulated_historical_data(ticker, start_date_str, end_date_str):
    """
    Simula la obtención de datos históricos para un ticker y rango de fechas.
    Esta función se usa como alternativa cuando la descarga de datos reales falla.
    """
    print(f"{Fore.YELLOW}Generando datos históricos SIMULADOS para {ticker} desde {start_date_str} hasta {end_date_str}...{Style.RESET_ALL}")
    start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    np.random.seed(hash(ticker) % (2**32 - 1)) 
    current_price = (ord(ticker[0]) % 10 + 1) * 100 
    
    data_list = []
    for single_date in date_range:
        if single_date.weekday() >= 5: 
            continue
        
        open_price = round(current_price + (np.random.rand() - 0.5) * 5, 2)
        high_price = round(open_price + np.random.rand() * 3, 2)
        low_price = round(open_price - np.random.rand() * 3, 2)
        close_price = round(current_price + (np.random.rand() - 0.5) * 6, 2) 
        volume = np.random.randint(100000, 1000000)

        data_list.append({
            'date': single_date,
            'open': open_price,
            'high': high_price,
            'low': low_price,
            'close': close_price,
            'Volume': volume 
        })
        current_price = close_price 

    df = pd.DataFrame(data_list)
    df.set_index('date', inplace=True)
    return df


# --- Lógica de Backtesting del Bot de Trading (Estrategia VWAP) ---

def run_backtest(ticker, start_date_str, end_date_str_display, initial_capital,
                 std_period, std_multiplier, trend_ema_period,
                 stop_loss_percent, take_profit_percent,
                 risk_free_rate_annual,
                 rsi_period=None, macd_fast_period=None, macd_slow_period=None, macd_signal_period=None,
                 avwap_anchor_date=None, vwap_short_period=None, vwap_long_period=None, # Nuevos parámetros VWAP Múltiples
                 use_trailing_sl=False, trailing_sl_percent=None, 
                 use_dynamic_position_sizing=False, risk_per_trade_percent=None, atr_risk_multiplier=None,
                 strategy_id="VWAP_EMA_TREND"):
    """
    Ejecuta un backtest de una estrategia de trading basada en VWAP.
    La lógica de compra/venta varía según el 'strategy_id'.
    """
    print(f"\n{Fore.CYAN}{'='*50}{Style.RESET_ALL}")
    print(f"{Fore.CYAN}Preparando backtest para {ticker} - Estrategia: {strategy_id} desde {start_date_str} hasta {end_date_str_display}...{Style.RESET_ALL}")
    print(f"{Fore.CYAN}{'='*50}{Style.RESET_ALL}\n")

    end_date_for_api = datetime.datetime.strptime(end_date_str_display, '%Y-%m-%d') + datetime.timedelta(days=1)
    end_date_for_api_str = end_date_for_api.strftime('%Y-%m-%d')

    data_source = "REALES"
    data = fetch_historical_data_real(ticker, start_date_str, end_date_for_api_str)
    
    if data.empty:
        print(f"\n{Fore.RED}🚨🚨🚨 ¡ATENCIÓN! FALLÓ LA DESCARGA DE DATOS REALES DE TODAS LAS FUENTES. 🚨🚨🚨{Style.RESET_ALL}")
        print(f"{Fore.YELLOW}El bot usará datos SIMULADOS para {strategy_id}. Los resultados NO son reales, son solo ilustrativos para la estrategia.{Style.RESET_ALL}")
        data = fetch_simulated_historical_data(ticker, start_date_str, end_date_str_display)
        data_source = "SIMULADOS"

    if data.empty:
        print(f"{Fore.RED}¡Error crítico! No se encontraron datos válidos (reales o simulados) para el backtest. El bot no puede continuar.{Style.RESET_ALL}")
        return None, None, None, data_source

    data = calculate_vwap_with_bands(data, std_period, std_multiplier)
    data['trend_ema'] = calculate_ema(data['close'], trend_ema_period)
    
    if rsi_period is not None:
        data['rsi'] = calculate_rsi(data['close'], rsi_period)
        
    if macd_fast_period is not None and macd_slow_period is not None and macd_signal_period is not None:
        macd_df = calculate_macd(data['close'], macd_fast_period, macd_slow_period, macd_signal_period)
        data = data.join(macd_df)

    if avwap_anchor_date is not None:
        data['avwap'] = calculate_anchored_vwap(data, avwap_anchor_date)

    if vwap_short_period is not None and vwap_long_period is not None:
        # Calcular VWAPs con diferentes períodos
        data['vwap_short'] = calculate_vwap_with_bands(data.copy(), std_period=vwap_short_period, std_multiplier=0)['vwap'] # std_multiplier=0 para no calcular bandas aquí
        data['vwap_long'] = calculate_vwap_with_bands(data.copy(), std_period=vwap_long_period, std_multiplier=0)['vwap']

    # Calcular ATR si se va a usar para tamaño de posición o cualquier otra cosa
    if use_dynamic_position_sizing:
        data['atr'] = calculate_atr(data)
        
    data.dropna(inplace=True)

    if data.empty:
        print(f"{Fore.YELLOW}¡Atención! No hay suficientes datos después de calcular los indicadores para la estrategia {strategy_id}. Intenta con un período de tiempo mayor o diferentes parámetros.{Style.RESET_ALL}")
        return None, None, None, data_source
        
    print(f"{Fore.GREEN}Datos procesados para {strategy_id}. {len(data)} entradas válidas para el backtest.{Style.RESET_ALL}\n")

    in_position = False
    cash = initial_capital
    shares = 0
    trade_log = []
    entry_price = 0
    current_trailing_stop = None # Para Trailing Stop-Loss

    data['buy_signal'] = np.nan
    data['sell_signal'] = np.nan
    data['portfolio_value'] = np.nan

    for i in range(1, len(data)):
        current_date = data.index[i]
        current_open = data['open'].iloc[i]
        current_high = data['high'].iloc[i]
        current_low = data['low'].iloc[i]
        current_close = data['close'].iloc[i]
        vwap = data['vwap'].iloc[i]
        vwap_upper_band = data['vwap_upper_band'].iloc[i]
        vwap_lower_band = data['vwap_lower_band'].iloc[i]
        trend_ema = data['trend_ema'].iloc[i]
        
        prev_close = data['close'].iloc[i-1]
        prev_vwap = data['vwap'].iloc[i-1]
        
        rsi_value = data['rsi'].iloc[i] if rsi_period is not None else None
        
        macd = data['macd'].iloc[i] if macd_fast_period is not None else None
        signal_line = data['signal_line'].iloc[i] if macd_fast_period is not None else None
        
        avwap_value = data['avwap'].iloc[i] if avwap_anchor_date is not None else None
        vwap_short = data['vwap_short'].iloc[i] if vwap_short_period is not None else None
        vwap_long = data['vwap_long'].iloc[i] if vwap_long_period is not None else None

        atr_value = data['atr'].iloc[i] if use_dynamic_position_sizing else None

        sold_this_day = False

        # --- Lógica de Venta (Stop-Loss / Take-Profit / Trailing SL / Estrategia) ---
        if in_position:
            # 1. Comprobar Trailing Stop-Loss (si está activo)
            if use_trailing_sl and current_trailing_stop is not None:
                if current_close <= current_trailing_stop:
                    cash += shares * current_close
                    profit_loss = (current_close - entry_price) * shares
                    trade_log.append({ 'date': current_date, 'type': 'SELL (Trailing SL)', 'price': current_close, 'shares': shares, 'profit_loss': profit_loss, 'portfolio_value': None })
                    data.loc[current_date, 'sell_signal'] = current_close
                    shares = 0
                    in_position = False
                    sold_this_day = True
                    current_trailing_stop = None 
                else: 
                    new_trailing_stop = current_close * (1 - trailing_sl_percent)
                    current_trailing_stop = max(current_trailing_stop, new_trailing_stop) 

            # 2. Comprobar Stop-Loss fijo (solo si no se activó Trailing SL)
            stop_loss_level = entry_price * (1 - stop_loss_percent)
            if not sold_this_day and current_close <= stop_loss_level:
                cash += shares * current_close
                profit_loss = (current_close - entry_price) * shares
                trade_log.append({ 'date': current_date, 'type': 'SELL (Stop-Loss)', 'price': current_close, 'shares': shares, 'profit_loss': profit_loss, 'portfolio_value': None })
                data.loc[current_date, 'sell_signal'] = current_close
                shares = 0
                in_position = False
                sold_this_day = True
                current_trailing_stop = None

            # 3. Comprobar Take-Profit (solo si no se activó SL/Trailing SL)
            take_profit_level = entry_price * (1 + take_profit_percent)
            if not sold_this_day and current_close >= take_profit_level:
                cash += shares * current_close
                profit_loss = (current_close - entry_price) * shares
                trade_log.append({ 'date': current_date, 'type': 'SELL (Take-Profit)', 'price': current_close, 'shares': shares, 'profit_loss': profit_loss, 'portfolio_value': None })
                data.loc[current_date, 'sell_signal'] = current_close
                shares = 0
                in_position = False
                sold_this_day = True
                current_trailing_stop = None

            # 4. Lógica de Venta Específica de la Estrategia (si no se activó ninguno de los anteriores)
            if not sold_this_day:
                if strategy_id in ["VWAP_EMA_TREND", "VWAP_MEAN_REVERSION", "VWAP_EMA_RSI_FILTER"]:
                    if (current_close < vwap and prev_close >= prev_vwap) or \
                       (current_close > vwap_upper_band and prev_close <= vwap_upper_band):
                        
                        if strategy_id == "VWAP_EMA_RSI_FILTER" and (rsi_value is not None and rsi_value < 70):
                            pass 
                        else:
                            cash += shares * current_close
                            profit_loss = (current_close - entry_price) * shares
                            trade_log.append({ 'date': current_date, 'type': f'SELL ({strategy_id})', 'price': current_close, 'shares': shares, 'profit_loss': profit_loss, 'portfolio_value': None })
                            data.loc[current_date, 'sell_signal'] = current_close
                            shares = 0
                            in_position = False
                            sold_this_day = True
                            current_trailing_stop = None
                
                elif strategy_id == "VWAP_MACD_MOMENTUM":
                    if (macd is not None and signal_line is not None and macd < signal_line and data['macd'].iloc[i-1] >= data['signal_line'].iloc[i-1]) or \
                       (current_close < vwap and prev_close >= prev_vwap):
                        cash += shares * current_close
                        profit_loss = (current_close - entry_price) * shares
                        trade_log.append({ 'date': current_date, 'type': f'SELL ({strategy_id})', 'price': current_close, 'shares': shares, 'profit_loss': profit_loss, 'portfolio_value': None })
                        data.loc[current_date, 'sell_signal'] = current_close
                        shares = 0
                        in_position = False
                        sold_this_day = True
                        current_trailing_stop = None
                
                elif strategy_id == "ANCHORED_VWAP":
                    if (avwap_value is not None and current_close < avwap_value and prev_close >= data['avwap'].iloc[i-1]):
                        cash += shares * current_close
                        profit_loss = (current_close - entry_price) * shares
                        trade_log.append({ 'date': current_date, 'type': f'SELL ({strategy_id})', 'price': current_close, 'shares': shares, 'profit_loss': profit_loss, 'portfolio_value': None })
                        data.loc[current_date, 'sell_signal'] = current_close
                        shares = 0
                        in_position = False
                        sold_this_day = True
                        current_trailing_stop = None

                elif strategy_id == "VWAP_CROSSOVER":
                    if (vwap_short is not None and vwap_long is not None and vwap_short < vwap_long and data['vwap_short'].iloc[i-1] >= data['vwap_long'].iloc[i-1]):
                        cash += shares * current_close
                        profit_loss = (current_close - entry_price) * shares
                        trade_log.append({ 'date': current_date, 'type': f'SELL ({strategy_id})', 'price': current_close, 'shares': shares, 'profit_loss': profit_loss, 'portfolio_value': None })
                        data.loc[current_date, 'sell_signal'] = current_close
                        shares = 0
                        in_position = False
                        sold_this_day = True
                        current_trailing_stop = None
                        
        # --- Lógica de Compra (solo si no se ha vendido hoy y no hay posición abierta) ---
        if not in_position and not sold_this_day:
            buy_condition_met = False
            
            if strategy_id == "VWAP_EMA_TREND":
                if (current_close > vwap and prev_close <= prev_vwap and current_close > trend_ema):
                    buy_condition_met = True
            
            elif strategy_id == "VWAP_MEAN_REVERSION":
                if (prev_close <= vwap_lower_band and current_close > vwap_lower_band) and \
                   (current_close > vwap or current_close > trend_ema):
                    buy_condition_met = True
            
            elif strategy_id == "VWAP_EMA_RSI_FILTER":
                if (current_close > vwap and prev_close <= prev_vwap and current_close > trend_ema) and \
                   (rsi_value is not None and rsi_value < 70):
                    buy_condition_met = True

            elif strategy_id == "VWAP_MACD_MOMENTUM":
                if (current_close > vwap and prev_close <= prev_vwap) and \
                   (macd is not None and signal_line is not None and macd > signal_line and data['macd'].iloc[i-1] <= data['signal_line'].iloc[i-1]):
                    buy_condition_met = True
            
            elif strategy_id == "ANCHORED_VWAP":
                if (avwap_value is not None and current_close > avwap_value and prev_close <= data['avwap'].iloc[i-1]):
                    buy_condition_met = True

            elif strategy_id == "VWAP_CROSSOVER":
                if (vwap_short is not None and vwap_long is not None and vwap_short > vwap_long and data['vwap_short'].iloc[i-1] <= data['vwap_long'].iloc[i-1]):
                    buy_condition_met = True

            if buy_condition_met:
                shares_to_buy = 0
                if use_dynamic_position_sizing and atr_value is not None and atr_value > 0:
                    risk_per_unit = atr_value * atr_risk_multiplier
                    if risk_per_unit > 0:
                        position_risk_amount = cash * risk_per_trade_percent
                        shares_to_buy = position_risk_amount / risk_per_unit
                        shares_to_buy = np.floor(shares_to_buy) 
                        
                        if shares_to_buy * current_close > cash:
                            shares_to_buy = np.floor(cash / current_close)
                else: 
                    shares_to_buy = cash / current_close

                if shares_to_buy > 0:
                    shares += shares_to_buy
                    cash -= (shares_to_buy * current_close) 
                    in_position = True
                    entry_price = current_close
                    if use_trailing_sl and trailing_sl_percent is not None:
                        current_trailing_stop = entry_price * (1 - trailing_sl_percent)

                    trade_log.append({ 'date': current_date, 'type': 'BUY', 'price': current_close, 'shares': shares_to_buy, 'portfolio_value': None })
                    data.loc[current_date, 'buy_signal'] = current_close
        
        data.loc[current_date, 'portfolio_value'] = cash + (shares * current_close)


    # --- Gestión de Posición Abierta al Final del Backtest ---
    if in_position:
        final_close = data['close'].iloc[-1]
        cash += shares * final_close
        profit_loss = (final_close - entry_price) * shares
        trade_log.append({ 'date': data.index[-1], 'type': 'SELL (Cierre final)', 'price': final_close, 'shares': shares, 'profit_loss': profit_loss, 'portfolio_value': None })
        data.loc[data.index[-1], 'sell_signal'] = final_close

    # --- Actualizar Valores del Portafolio en el Registro de Operaciones ---
    df_portfolio_value = data[['portfolio_value']].dropna()

    if not df_portfolio_value.empty:
        for trade in trade_log:
            if trade['date'] in df_portfolio_value.index:
                trade['portfolio_value'] = df_portfolio_value.loc[trade['date']]['portfolio_value']
            else:
                trade['portfolio_value'] = initial_capital
    else:
        print(f"{Fore.YELLOW}Advertencia ({strategy_id}): No se pudo construir la serie de valor del portafolio. Las métricas pueden ser imprecisas.{Style.RESET_ALL}")

    df_trade_log = pd.DataFrame(trade_log)
    if df_trade_log.empty:
        df_trade_log = pd.DataFrame(columns=['date', 'type', 'price', 'shares', 'profit_loss', 'portfolio_value'])


    # --- Cálculo de Métricas de Rendimiento ---
    if not df_portfolio_value.empty:
        final_capital = df_portfolio_value['portfolio_value'].iloc[-1]
    else:
        final_capital = initial_capital 

    total_return = ((final_capital - initial_capital) / initial_capital) * 100

    max_drawdown = 0
    if not df_portfolio_value.empty:
        portfolio_values_series = df_portfolio_value['portfolio_value']
        peak = portfolio_values_series.expanding(min_periods=1).max()
        drawdown = (portfolio_values_series - peak) / peak
        max_drawdown = drawdown.min() * 100 if not drawdown.empty else 0

    winning_trades = 0
    losing_trades = 0
    total_trades = 0
    win_rate = 0
    if not df_trade_log.empty and 'profit_loss' in df_trade_log.columns:
        winning_trades = df_trade_log[df_trade_log['profit_loss'] > 0].shape[0]
        losing_trades = df_trade_log[df_trade_log['profit_loss'] < 0].shape[0]
        total_trades = winning_trades + losing_trades 
        if total_trades > 0:
            win_rate = (winning_trades / total_trades) * 100

    daily_returns = data['portfolio_value'].pct_change().dropna()
    sharpe_ratio = np.nan
    if not daily_returns.empty and daily_returns.std() != 0:
        annualized_return = daily_returns.mean() * 252
        annualized_std = daily_returns.std() * np.sqrt(252)
        sharpe_ratio = (annualized_return - risk_free_rate_annual) / annualized_std

    metrics = {
        'strategy_name': strategy_id,
        'initial_capital': initial_capital,
        'final_capital': round(final_capital, 2),
        'total_return_percent': round(total_return, 2),
        'total_trades': total_trades,
        'winning_trades': winning_trades,
        'losing_trades': losing_trades,
        'win_rate_percent': round(win_rate, 2),
        'max_drawdown_percent': round(max_drawdown, 2),
        'sharpe_ratio': round(sharpe_ratio, 2) if not np.isnan(sharpe_ratio) else np.nan,
    }

    return metrics, df_trade_log, data, data_source


# --- Función para Visualización (individual con Plotly) ---

def plot_backtest_results(data, ticker, initial_capital, strategy_name, 
                          vwap_std_period_for_plot, trend_ema_period_for_plot, 
                          rsi_period_for_plot=None, macd_periods_for_plot=None,
                          avwap_anchor_date_for_plot=None, vwap_crossover_periods_for_plot=None): 
    """
    Genera un gráfico interactivo con Plotly de los resultados del backtest para una sola estrategia,
    mostrando el precio de cierre, el VWAP, sus bandas, la EMA de tendencia,
    las señales de compra/venta y la evolución del portafolio, incluyendo RSI y MACD en subplots si se usan.
    """
    # Determinar cuántos subplots necesitamos
    rows = 2 # Precios + Portafolio
    row_heights = [0.5, 0.2] # Proporciones de altura
    row_titles = ['Precios e Indicadores', 'Valor del Portafolio']

    if rsi_period_for_plot is not None and 'rsi' in data.columns and data['rsi'].notna().any():
        rows += 1
        row_heights.append(0.15)
        row_titles.append('RSI')
    if macd_periods_for_plot is not None and 'macd' in data.columns and data['macd'].notna().any():
        rows += 1
        row_heights.append(0.15)
        row_titles.append(f'MACD ({macd_periods_for_plot[0]}, {macd_periods_for_plot[1]}, {macd_periods_for_plot[2]})')


    fig = make_subplots(rows=rows, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, # Reducir espacio vertical
                        row_titles=row_titles,
                        row_heights=row_heights)

    # --- Gráfico Superior: Precios e Indicadores ---
    # Candlestick chart para precios
    fig.add_trace(go.Candlestick(x=data.index,
                                 open=data['open'],
                                 high=data['high'],
                                 low=data['low'],
                                 close=data['close'],
                                 name='Velas',
                                 increasing_line_color='#2ECC71', # Verde esmeralda
                                 decreasing_line_color='#E74C3C'), # Rojo
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=data.index, y=data['vwap'], mode='lines', name='VWAP',
                             line=dict(color='#3498DB', width=2.0, dash='dash')), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['trend_ema'], mode='lines', name=f'EMA Tendencia ({trend_ema_period_for_plot})',
                             line=dict(color='#E74C3C', width=1.5, dash='dot')), row=1, col=1)

    # Añadir AVWAP si la estrategia lo usa
    if avwap_anchor_date_for_plot is not None and 'avwap' in data.columns and data['avwap'].notna().any():
        fig.add_trace(go.Scatter(x=data.index, y=data['avwap'], mode='lines', name=f'AVWAP (Anclado a {avwap_anchor_date_for_plot})',
                                 line=dict(color='#9B59B6', width=2.0, dash='solid')), row=1, col=1)
    
    # Añadir VWAP corto y largo si la estrategia de crossover lo usa
    if vwap_crossover_periods_for_plot is not None and 'vwap_short' in data.columns and data['vwap_short'].notna().any():
        fig.add_trace(go.Scatter(x=data.index, y=data['vwap_short'], mode='lines', name=f'VWAP Corto ({vwap_crossover_periods_for_plot[0]})',
                                 line=dict(color='#FFA07A', width=1.5)), row=1, col=1) # Color naranja claro
        fig.add_trace(go.Scatter(x=data.index, y=data['vwap_long'], mode='lines', name=f'VWAP Largo ({vwap_crossover_periods_for_plot[1]})',
                                 line=dict(color='#B0C4DE', width=1.5, dash='dash')), row=1, col=1) # Color azul grisáceo

    # Bandas VWAP (relleno entre ellas)
    fig.add_trace(go.Scatter(x=data.index, y=data['vwap_upper_band'], mode='lines', name=f'Banda Superior VWAP ({vwap_std_period_for_plot} períodos)',
                             line=dict(color='#F1C40F', width=1, dash='dot'), showlegend=True), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['vwap_lower_band'], mode='lines', name=f'Banda Inferior VWAP ({vwap_std_period_for_plot} períodos)',
                             line=dict(color='#F1C40F', width=1, dash='dot'), fill='tonexty', fillcolor='rgba(241,196,15,0.2)', showlegend=True), row=1, col=1)
    
    # Señales de compra
    buy_signals = data[data['buy_signal'].notna()]
    if not buy_signals.empty:
        fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['buy_signal'], mode='markers', name='Compra',
                                 marker=dict(symbol='triangle-up', size=12, color='#2ECC71', line=dict(width=1, color='DarkSlateGrey'))), row=1, col=1)
    # Señales de venta
    sell_signals = data[data['sell_signal'].notna()]
    if not sell_signals.empty:
        fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['sell_signal'], mode='markers', name='Venta',
                                 marker=dict(symbol='triangle-down', size=12, color='#E74C3C', line=dict(width=1, color='DarkSlateGrey'))), row=1, col=1)

    fig.update_yaxes(title_text='Precio (USD)', row=1, col=1)

    # --- Subplot de Portafolio (siempre presente después de precios) ---
    current_row_idx = 2
    if 'portfolio_value' in data.columns and data['portfolio_value'].notna().any():
        fig.add_trace(go.Scatter(x=data.index, y=data['portfolio_value'], mode='lines', name='Valor del Portafolio',
                                 line=dict(color='#9B59B6', width=2.5)), row=current_row_idx, col=1)
        fig.add_trace(go.Scatter(x=[data.index.min(), data.index.max()], y=[initial_capital, initial_capital], mode='lines',
                                 name='Capital Inicial', line=dict(color='#7F8C8D', width=1, dash='dash'), showlegend=True), row=current_row_idx, col=1)
        fig.update_yaxes(title_text='Valor (USD)', row=current_row_idx, col=1)
    else:
        fig.add_trace(go.Scatter(x=[data.index.min(), data.index.max()], y=[initial_capital, initial_capital], mode='lines',
                                 name='Capital Inicial', line=dict(color='#7F8C8D', width=1, dash='dash'), showlegend=True), row=current_row_idx, col=1)
        fig.add_annotation(dict(xref='paper', yref='paper', x=0.5, y=0.5,
                                text='No hay datos de valor de portafolio para graficar',
                                showarrow=False, font=dict(size=12, color='gray')), row=current_row_idx, col=1)
    current_row_idx += 1

    # --- Subplot de RSI (si aplica) ---
    if rsi_period_for_plot is not None and 'rsi' in data.columns and data['rsi'].notna().any():
        fig.add_trace(go.Scatter(x=data.index, y=data['rsi'], mode='lines', name=f'RSI ({rsi_period_for_plot})',
                                 line=dict(color='#E67E22', width=1.5)), row=current_row_idx, col=1)
        fig.add_trace(go.Scatter(x=[data.index.min(), data.index.max()], y=[70, 70], mode='lines',
                                 name='Sobrecompra (70)', line=dict(color='#C0392B', width=1, dash='dash')), row=current_row_idx, col=1)
        fig.add_trace(go.Scatter(x=[data.index.min(), data.index.max()], y=[30, 30], mode='lines',
                                 name='Sobreventa (30)', line=dict(color='#28B463', width=1, dash='dash')), row=current_row_idx, col=1)
        fig.update_yaxes(title_text='RSI', range=[0, 100], row=current_row_idx, col=1)
        current_row_idx += 1

    # --- Subplot de MACD (si aplica) ---
    if macd_periods_for_plot is not None and 'macd' in data.columns and data['macd'].notna().any():
        fig.add_trace(go.Scatter(x=data.index, y=data['macd'], mode='lines', name=f'MACD',
                                 line=dict(color='#3498DB', width=1.5)), row=current_row_idx, col=1)
        fig.add_trace(go.Scatter(x=data.index, y=data['signal_line'], mode='lines', name=f'Señal MACD',
                                 line=dict(color='#E74C3C', width=1.5, dash='dot')), row=current_row_idx, col=1)
        fig.add_trace(go.Bar(x=data.index, y=data['macd_histogram'], name='Histograma MACD',
                             marker_color=np.where(data['macd_histogram'] >= 0, '#2ECC71', '#C0392B'), opacity=0.7), row=current_row_idx, col=1)
        fig.update_yaxes(title_text='MACD', row=current_row_idx, col=1)
        current_row_idx += 1


    fig.update_layout(
        title_text=f'Backtest de Estrategia {strategy_name} para {ticker}',
        title_font_size=22,
        title_font_color='#2C3E50',
        height=500 + (rows - 2) * 200, # Ajusta la altura total de la figura dinámicamente
        xaxis_rangeslider_visible=False, # Se puede activar interactivamente
        hovermode='x unified',
        template='plotly_dark'
    )
    
    # Ajustar el eje X para el último subplot
    fig.update_xaxes(title_text='Fecha', row=rows, col=1)

    fig.show()

# --- Función para Visualización Comparativa (con Plotly) ---

def plot_comparison_results(all_results, ticker, initial_capital):
    """
    Genera un gráfico interactivo con Plotly comparativo de la evolución del valor del portafolio para todas las estrategias.
    """
    fig = go.Figure()
    
    colors = ['#3498DB', '#2ECC71', '#9B59B6', '#E67E22', '#F1C40F', '#7F8C8D', '#FF6347', '#4682B4'] 
    
    if all_results:
        benchmark_data = all_results[0]['historical_data_with_signals'] 
        if not benchmark_data.empty:
            first_close_price = benchmark_data['close'].iloc[0]
            buy_and_hold_value = (benchmark_data['close'] / first_close_price) * initial_capital
            fig.add_trace(go.Scatter(x=benchmark_data.index, y=buy_and_hold_value, mode='lines',
                                     name='Buy and Hold (Benchmark)', line=dict(color='gray', dash='dashdot', width=2)))

    for i, result in enumerate(all_results):
        strategy_name = result['strategy_config']['name']
        data = result['historical_data_with_signals']
        
        if 'portfolio_value' in data.columns and data['portfolio_value'].notna().any():
            fig.add_trace(go.Scatter(x=data.index, y=data['portfolio_value'], mode='lines',
                                     name=f'{strategy_name}', line=dict(color=colors[i % len(colors)], width=2.5)))
        else:
            fig.add_annotation(dict(xref='paper', yref='paper', x=0.05, y=0.9 - (i+1)*0.05, 
                                    text=f'No hay datos de portafolio para {strategy_name}',
                                    showarrow=False, font=dict(size=12, color=colors[i % len(colors)])))

    fig.add_trace(go.Scatter(x=[benchmark_data.index.min(), benchmark_data.index.max()], 
                             y=[initial_capital, initial_capital], mode='lines',
                             name='Capital Inicial', line=dict(color='#7F8C8D', dash='dash', width=1)))

    fig.update_layout(
        title_text=f'Comparación de Estrategias para {ticker}',
        title_font_size=22,
        title_font_color='#2C3E50',
        xaxis_title='Fecha',
        yaxis_title='Valor del Portafolio (USD)',
        hovermode='x unified',
        template='plotly_dark',
        height=700
    )
    
    fig.show()

def plot_profit_loss_distribution(trade_logs, strategy_names): # initial_capital no es necesario aquí
    """
    Genera un histograma interactivo de Plotly para la distribución de ganancias/pérdidas por operación
    para cada estrategia.
    """
    if not trade_logs:
        print(f"{Fore.YELLOW}No hay datos de operaciones para graficar la distribución de ganancias/pérdidas.{Style.RESET_ALL}")
        return

    fig = go.Figure()
    colors = ['#3498DB', '#2ECC71', '#9B59B6', '#E67E22', '#F1C40F', '#7F8C8D', '#FF6347', '#4682B4']

    for i, (strategy_name, df_trade_log) in enumerate(zip(strategy_names, trade_logs)):
        if not df_trade_log.empty and 'profit_loss' in df_trade_log.columns and not df_trade_log['profit_loss'].dropna().empty:
            fig.add_trace(go.Histogram(x=df_trade_log['profit_loss'].dropna(), name=strategy_name,
                                       xbins=dict(start=df_trade_log['profit_loss'].min() - 1, 
                                                  end=df_trade_log['profit_loss'].max() + 1, 
                                                  size=(df_trade_log['profit_loss'].max() - df_trade_log['profit_loss'].min() + 2) / 50), # 50 bins dinámicos
                                       marker_color=colors[i % len(colors)], opacity=0.7))
        else:
            print(f"{Fore.YELLOW}No hay operaciones válidas para {strategy_name} para graficar la distribución.{Style.RESET_ALL}")

    fig.update_layout(
        title_text='Distribución de Ganancias/Pérdidas por Operación',
        title_font_size=22,
        title_font_color='#2C3E50',
        xaxis_title='Ganancia/Pérdida por Operación (USD)',
        yaxis_title='Frecuencia',
        barmode='overlay', 
        hovermode='x unified',
        template='plotly_dark',
        height=600
    )
    fig.show()

def export_results_to_csv(all_strategy_results, comparison_metrics_df, ticker_symbol):
    """
    Exporta las métricas de comparación y los registros de operaciones a archivos CSV.
    """
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if not comparison_metrics_df.empty:
        comparison_filename = f"{ticker_symbol}_comparison_metrics_{timestamp}.csv"
        comparison_metrics_df.to_csv(comparison_filename, index=True)
        print(f"\n{Fore.GREEN}✅ Métricas comparativas exportadas a: {comparison_filename}{Style.RESET_ALL}")
    else:
        print(f"\n{Fore.YELLOW}❌ No hay métricas comparativas para exportar.{Style.RESET_ALL}")

    if all_strategy_results:
        for result in all_strategy_results:
            strategy_name_for_file = result['strategy_config']['name'].replace(" ", "_").replace(":", "").replace("(", "").replace(")", "").replace("-", "_")
            trade_log_filename = f"{ticker_symbol}_{strategy_name_for_file}_trade_log_{timestamp}.csv"
            if not result['trade_log_df'].empty:
                result['trade_log_df'].to_csv(trade_log_filename, index=False)
                print(f"{Fore.GREEN}✅ Registro de operaciones de '{result['strategy_config']['name']}' exportado a: {trade_log_filename}{Style.RESET_ALL}")
            else:
                print(f"{Fore.YELLOW}❌ No hay registro de operaciones para '{result['strategy_config']['name']}' para exportar.{Style.RESET_ALL}")
    else:
        print(f"\n{Fore.YELLOW}❌ No hay resultados de estrategias para exportar registros de operaciones.{Style.RESET_ALL}")


# --- Ejecución Principal en Jupyter Labs ---

if __name__ == '__main__':
    # Parámetros por defecto para la estrategia (AJUSTADOS)
    DEFAULT_VWAP_STD_PERIOD = 20
    DEFAULT_VWAP_STD_MULTIPLIER = 1.0 
    DEFAULT_TREND_EMA_PERIOD = 50     
    DEFAULT_STOP_LOSS_PERCENT = 0.015
    DEFAULT_TAKE_PROFIT_PERCENT = 0.03
    DEFAULT_RISK_FREE_RATE_ANNUAL = 0.02 # 2% anual
    DEFAULT_RSI_PERIOD = 14
    DEFAULT_MACD_FAST_PERIOD = 12
    DEFAULT_MACD_SLOW_PERIOD = 26
    DEFAULT_MACD_SIGNAL_PERIOD = 9
    DEFAULT_AVWAP_ANCHOR_DATE = (datetime.date.today() - datetime.timedelta(days=365)).strftime('%Y-%m-%d') # Un año atrás
    DEFAULT_VWAP_CROSSOVER_SHORT = 10
    DEFAULT_VWAP_CROSSOVER_LONG = 50


    # Parámetros por defecto para gestión de riesgo avanzada
    DEFAULT_TRAILING_SL_PERCENT = 0.01 # 1% Trailing Stop Loss
    DEFAULT_RISK_PER_TRADE_PERCENT = 0.01 # Arriesgar 1% del capital por trade
    DEFAULT_ATR_RISK_MULTIPLIER = 2.0 # Stop Loss a 2x ATR de la entrada


    print(f"{Fore.MAGENTA}👋 Bienvenido/a al Bot de Trading VWAP Avanzado y Amigable!{Style.RESET_ALL}")
    MY_TICKER = input(f"{Fore.CYAN}Por favor, introduce el ticker del activo (ej. TSLA, AAPL, MSFT): {Style.RESET_ALL}").upper()
    INITIAL_CAPITAL = float(input(f"{Fore.CYAN}Capital inicial para el backtest (por defecto 10000): {Style.RESET_ALL}"))
    RISK_FREE_RATE_ANNUAL = float(input(f"{Fore.CYAN}Tasa de Retorno Anual Libre de Riesgo (por defecto {DEFAULT_RISK_FREE_RATE_ANNUAL}, ej. 0.02 para 2%): {Style.RESET_ALL}"))
    
    use_trailing_sl_global = input(f"{Fore.CYAN}¿Deseas activar un Trailing Stop-Loss para todas las estrategias? (s/n, por defecto n): {Style.RESET_ALL}").lower() == 's'
    trailing_sl_percent_global = DEFAULT_TRAILING_SL_PERCENT
    if use_trailing_sl_global:
        trailing_sl_percent_global = float(input(f"{Fore.CYAN}  Porcentaje para el Trailing Stop-Loss (por defecto {DEFAULT_TRAILING_SL_PERCENT}, ej. 0.01 para 1%): {Style.RESET_ALL}"))

    use_dynamic_position_sizing_global = input(f"{Fore.CYAN}¿Deseas activar el tamaño de posición dinámico (basado en ATR) para todas las estrategias? (s/n, por defecto n): {Style.RESET_ALL}").lower() == 's'
    risk_per_trade_percent_global = DEFAULT_RISK_PER_TRADE_PERCENT
    atr_risk_multiplier_global = DEFAULT_ATR_RISK_MULTIPLIER
    if use_dynamic_position_sizing_global:
        risk_per_trade_percent_global = float(input(f"{Fore.CYAN}  Porcentaje de capital a arriesgar por trade (por defecto {DEFAULT_RISK_PER_TRADE_PERCENT}, ej. 0.01 para 1%): {Style.RESET_ALL}"))
        atr_risk_multiplier_global = float(input(f"{Fore.CYAN}  Multiplicador ATR para el riesgo por trade (por defecto {DEFAULT_ATR_RISK_MULTIPLIER}, ej. 2.0 para 2x ATR): {Style.RESET_ALL}"))

    END_DATE_DISPLAY = datetime.date.today()
    END_DATE_FOR_API = END_DATE_DISPLAY + datetime.timedelta(days=1) 
    START_DATE = END_DATE_DISPLAY - datetime.timedelta(days=10*365) # Últimos 10 años por defecto
    
    START_DATE_STR = START_DATE.strftime('%Y-%m-%d')
    END_DATE_DISPLAY_STR = END_DATE_DISPLAY.strftime('%Y-%m-%d')
    END_DATE_FOR_API_STR = END_DATE_FOR_API.strftime('%Y-%m-%d')

    print(f"\n{Fore.MAGENTA}Iniciando backtest para {MY_TICKER} con múltiples estrategias VWAP...{Style.RESET_ALL}")
    print(f"{Fore.MAGENTA}Analizando rango de fechas: {START_DATE_STR} a {END_DATE_DISPLAY_STR}{Style.RESET_ALL}\n")
    print(f"{Fore.YELLOW}⚠️ ADVERTENCIA: Se intentará descargar datos hasta HOY (último día de operación disponible). Si el mercado aún está abierto o los datos no están completamente procesados por las fuentes de datos, la descarga puede fallar o ser inexacta.{Style.RESET_ALL}")
    print(f"{Fore.WHITE}{'-'*70}{Style.RESET_ALL}")

    # --- Definición BASE de las Estrategias a Comparar ---
    base_strategies_to_test = [
        {
            'name': 'Estrategia 1: VWAP + EMA (Tendencia)',
            'id': 'VWAP_EMA_TREND',
            'std_period': DEFAULT_VWAP_STD_PERIOD,
            'std_multiplier': DEFAULT_VWAP_STD_MULTIPLIER,
            'trend_ema_period': DEFAULT_TREND_EMA_PERIOD,
            'stop_loss_percent': DEFAULT_STOP_LOSS_PERCENT,
            'take_profit_percent': DEFAULT_TAKE_PROFIT_PERCENT,
            'rsi_period': None,
            'macd_periods': None,
            'avwap_anchor_date': None,
            'vwap_crossover_periods': None
        },
        {
            'name': 'Estrategia 2: VWAP (Reversión a la Media)',
            'id': 'VWAP_MEAN_REVERSION',
            'std_period': 20,
            'std_multiplier': 1.5,
            'trend_ema_period': 50,
            'stop_loss_percent': 0.02, 
            'take_profit_percent': 0.04,
            'rsi_period': None,
            'macd_periods': None,
            'avwap_anchor_date': None,
            'vwap_crossover_periods': None
        },
        {
            'name': 'Estrategia 3: VWAP + EMA + RSI (Tendencia Filtrada)',
            'id': 'VWAP_EMA_RSI_FILTER',
            'std_period': 20,
            'std_multiplier': 1.0,
            'trend_ema_period': 50,
            'stop_loss_percent': 0.015,
            'take_profit_percent': 0.03,
            'rsi_period': DEFAULT_RSI_PERIOD,
            'macd_periods': None,
            'avwap_anchor_date': None,
            'vwap_crossover_periods': None
        },
        {
            'name': 'Estrategia 4: VWAP + MACD (Momentum)',
            'id': 'VWAP_MACD_MOMENTUM',
            'std_period': DEFAULT_VWAP_STD_PERIOD,
            'std_multiplier': DEFAULT_VWAP_STD_MULTIPLIER,
            'trend_ema_period': DEFAULT_TREND_EMA_PERIOD, 
            'stop_loss_percent': DEFAULT_STOP_LOSS_PERCENT,
            'take_profit_percent': DEFAULT_TAKE_PROFIT_PERCENT,
            'rsi_period': None,
            'macd_periods': (DEFAULT_MACD_FAST_PERIOD, DEFAULT_MACD_SLOW_PERIOD, DEFAULT_MACD_SIGNAL_PERIOD),
            'avwap_anchor_date': None,
            'vwap_crossover_periods': None
        },
        {
            'name': 'Estrategia 5: Anchored VWAP (AVWAP)', # Nueva estrategia
            'id': 'ANCHORED_VWAP',
            'std_period': DEFAULT_VWAP_STD_PERIOD, # Se usa para las bandas estándar, no para el AVWAP
            'std_multiplier': DEFAULT_VWAP_STD_MULTIPLIER,
            'trend_ema_period': DEFAULT_TREND_EMA_PERIOD,
            'stop_loss_percent': DEFAULT_STOP_LOSS_PERCENT,
            'take_profit_percent': DEFAULT_TAKE_PROFIT_PERCENT,
            'rsi_period': None,
            'macd_periods': None,
            'avwap_anchor_date': DEFAULT_AVWAP_ANCHOR_DATE,
            'vwap_crossover_periods': None
        },
        {
            'name': 'Estrategia 6: Cruce de VWAP Múltiples', # Nueva estrategia
            'id': 'VWAP_CROSSOVER',
            'std_period': DEFAULT_VWAP_STD_PERIOD,
            'std_multiplier': DEFAULT_VWAP_STD_MULTIPLIER,
            'trend_ema_period': DEFAULT_TREND_EMA_PERIOD,
            'stop_loss_percent': DEFAULT_STOP_LOSS_PERCENT,
            'take_profit_percent': DEFAULT_TAKE_PROFIT_PERCENT,
            'rsi_period': None,
            'macd_periods': None,
            'avwap_anchor_date': None,
            'vwap_crossover_periods': (DEFAULT_VWAP_CROSSOVER_SHORT, DEFAULT_VWAP_CROSSOVER_LONG)
        }
    ]

    final_strategies_to_run = []
    
    customize_all = input(f"\n{Fore.CYAN}¿Quieres correr todas las estrategias con sus parámetros predefinidos, o prefieres personalizar una o más? (predeterminado/personalizar): {Style.RESET_ALL}").lower()

    if customize_all == 'personalizar':
        for i, strat_config in enumerate(base_strategies_to_test):
            run_this_strategy = input(f"\n{Fore.CYAN}¿Quieres ejecutar '{strat_config['name']}'? (s/n): {Style.RESET_ALL}").lower()
            if run_this_strategy == 's':
                use_strat_defaults = input(f"{Fore.CYAN}¿Quieres usar los parámetros predefinidos para '{strat_config['name']}'? (s/n): {Style.RESET_ALL}").lower()
                if use_strat_defaults == 'n':
                    print(f"\n{Fore.YELLOW}--- Personalizando parámetros para '{strat_config['name']}' ---{Style.RESET_ALL}")
                    strat_config['std_period'] = int(input(f"{Fore.CYAN}  Período para las Bandas VWAP (actual {strat_config['std_period']}): {Style.RESET_ALL}"))
                    strat_config['std_multiplier'] = float(input(f"{Fore.CYAN}  Multiplicador de las Bandas VWAP (actual {strat_config['std_multiplier']}): {Style.RESET_ALL}"))
                    strat_config['trend_ema_period'] = int(input(f"{Fore.CYAN}  Período de la EMA de Tendencia (actual {strat_config['trend_ema_period']}): {Style.RESET_ALL}"))
                    strat_config['stop_loss_percent'] = float(input(f"{Fore.CYAN}  Porcentaje de Stop-Loss (actual {strat_config['stop_loss_percent']}): {Style.RESET_ALL}"))
                    strat_config['take_profit_percent'] = float(input(f"{Fore.CYAN}  Porcentaje de Take-Profit (actual {strat_config['take_profit_percent']}): {Style.RESET_ALL}"))
                    
                    # Personalizar RSI
                    if strat_config['rsi_period'] is not None:
                        strat_config['rsi_period'] = int(input(f"{Fore.CYAN}  Período del RSI (actual {strat_config['rsi_period']}): {Style.RESET_ALL}"))
                    else: 
                        add_rsi = input(f"{Fore.CYAN}  ¿Quieres añadir un filtro RSI a esta estrategia? (s/n): {Style.RESET_ALL}").lower()
                        if add_rsi == 's':
                            strat_config['rsi_period'] = int(input(f"{Fore.CYAN}  Período del RSI (por defecto {DEFAULT_RSI_PERIOD}): {Style.RESET_ALL}"))
                    
                    # Personalizar MACD
                    if strat_config['macd_periods'] is not None:
                        print(f"{Fore.CYAN}  Períodos MACD (actual: {strat_config['macd_periods'][0]}, {strat_config['macd_periods'][1]}, {strat_config['macd_periods'][2]}):{Style.RESET_ALL}")
                        macd_fast = int(input(f"{Fore.CYAN}    Período Rápido (fast_period): {Style.RESET_ALL}"))
                        macd_slow = int(input(f"{Fore.CYAN}    Período Lento (slow_period): {Style.RESET_ALL}"))
                        macd_signal = int(input(f"{Fore.CYAN}    Período de Señal (signal_period): {Style.RESET_ALL}"))
                        strat_config['macd_periods'] = (macd_fast, macd_slow, macd_signal)
                    else:
                        add_macd = input(f"{Fore.CYAN}  ¿Quieres añadir un filtro MACD a esta estrategia? (s/n): {Style.RESET_ALL}").lower()
                        if add_macd == 's':
                            print(f"{Fore.CYAN}  Períodos MACD (por defecto: {DEFAULT_MACD_FAST_PERIOD}, {DEFAULT_MACD_SLOW_PERIOD}, {DEFAULT_MACD_SIGNAL_PERIOD}):{Style.RESET_ALL}")
                            macd_fast = int(input(f"{Fore.CYAN}    Período Rápido (fast_period): {Style.RESET_ALL}"))
                            macd_slow = int(input(f"{Fore.CYAN}    Período Lento (slow_period): {Style.RESET_ALL}"))
                            macd_signal = int(input(f"{Fore.CYAN}    Período de Señal (signal_period): {Style.RESET_ALL}"))
                            strat_config['macd_periods'] = (macd_fast, macd_slow, macd_signal)

                    # Personalizar Anchored VWAP
                    if strat_config['avwap_anchor_date'] is not None:
                         strat_config['avwap_anchor_date'] = input(f"{Fore.CYAN}  Fecha de Anclaje para AVWAP (actual {strat_config['avwap_anchor_date']}, formato YYYY-MM-DD): {Style.RESET_ALL}")
                    else:
                         add_avwap = input(f"{Fore.CYAN}  ¿Quieres añadir Anchored VWAP a esta estrategia? (s/n): {Style.RESET_ALL}").lower()
                         if add_avwap == 's':
                             strat_config['avwap_anchor_date'] = input(f"{Fore.CYAN}  Fecha de Anclaje para AVWAP (por defecto {DEFAULT_AVWAP_ANCHOR_DATE}, formato YYYY-MM-DD): {Style.RESET_ALL}")
                    
                    # Personalizar VWAP Crossover
                    if strat_config['vwap_crossover_periods'] is not None:
                        print(f"{Fore.CYAN}  Períodos para Cruce de VWAP Múltiples (actual: Corto {strat_config['vwap_crossover_periods'][0]}, Largo {strat_config['vwap_crossover_periods'][1]}):{Style.RESET_ALL}")
                        vwap_short_p = int(input(f"{Fore.CYAN}    Período VWAP Corto: {Style.RESET_ALL}"))
                        vwap_long_p = int(input(f"{Fore.CYAN}    Período VWAP Largo: {Style.RESET_ALL}"))
                        strat_config['vwap_crossover_periods'] = (vwap_short_p, vwap_long_p)
                    else:
                        add_vwap_crossover = input(f"{Fore.CYAN}  ¿Quieres añadir Cruce de VWAP Múltiples a esta estrategia? (s/n): {Style.RESET_ALL}").lower()
                        if add_vwap_crossover == 's':
                            print(f"{Fore.CYAN}  Períodos para Cruce de VWAP Múltiples (por defecto: Corto {DEFAULT_VWAP_CROSSOVER_SHORT}, Largo {DEFAULT_VWAP_CROSSOVER_LONG}):{Style.RESET_ALL}")
                            vwap_short_p = int(input(f"{Fore.CYAN}    Período VWAP Corto: {Style.RESET_ALL}"))
                            vwap_long_p = int(input(f"{Fore.CYAN}    Período VWAP Largo: {Style.RESET_ALL}"))
                            strat_config['vwap_crossover_periods'] = (vwap_short_p, vwap_long_p)

                final_strategies_to_run.append(strat_config)
    else: 
        final_strategies_to_run = base_strategies_to_test

    all_strategy_results = []
    all_trade_logs_for_comparison = [] 
    all_strategy_names_for_comparison = [] 

    for strategy_config in final_strategies_to_run:
        print(f"\n{Fore.BLUE}{'='*60}{Style.RESET_ALL}")
        print(f"{Fore.BLUE}EJECUTANDO ESTRATEGIA: {strategy_config['name']}{Style.RESET_ALL}")
        print(f"{Fore.BLUE}{'='*60}{Style.RESET_ALL}\n")

        metrics, trade_log_df, historical_data_with_signals, data_source = run_backtest(
            MY_TICKER, START_DATE_STR, END_DATE_DISPLAY_STR,
            initial_capital=INITIAL_CAPITAL,
            std_period=strategy_config['std_period'],
            std_multiplier=strategy_config['std_multiplier'],
            trend_ema_period=strategy_config['trend_ema_period'],
            stop_loss_percent=strategy_config['stop_loss_percent'],
            take_profit_percent=strategy_config['take_profit_percent'],
            risk_free_rate_annual=RISK_FREE_RATE_ANNUAL,
            rsi_period=strategy_config['rsi_period'],
            macd_fast_period=strategy_config['macd_periods'][0] if strategy_config['macd_periods'] else None,
            macd_slow_period=strategy_config['macd_periods'][1] if strategy_config['macd_periods'] else None,
            macd_signal_period=strategy_config['macd_periods'][2] if strategy_config['macd_periods'] else None,
            avwap_anchor_date=strategy_config['avwap_anchor_date'],
            vwap_short_period=strategy_config['vwap_crossover_periods'][0] if strategy_config['vwap_crossover_periods'] else None,
            vwap_long_period=strategy_config['vwap_crossover_periods'][1] if strategy_config['vwap_crossover_periods'] else None,
            use_trailing_sl=use_trailing_sl_global,
            trailing_sl_percent=trailing_sl_percent_global,
            use_dynamic_position_sizing=use_dynamic_position_sizing_global,
            risk_per_trade_percent=risk_per_trade_percent_global,
            atr_risk_multiplier=atr_risk_multiplier_global,
            strategy_id=strategy_config['id']
        )
        
        if metrics:
            result_entry = {
                'metrics': metrics,
                'trade_log_df': trade_log_df,
                'historical_data_with_signals': historical_data_with_signals,
                'data_source': data_source,
                'strategy_config': strategy_config
            }
            all_strategy_results.append(result_entry)
            all_trade_logs_for_comparison.append(trade_log_df)
            all_strategy_names_for_comparison.append(strategy_config['name'])

            print(f"\n{Fore.GREEN}{'='*20} RESULTADOS INDIVIDUALES DEL BACKTEST - {strategy_config['name']} {'='*20}{Style.RESET_ALL}")
            print(f"{Fore.BLUE}Ticker Analizado: {MY_TICKER}{Style.RESET_ALL}")
            print(f"{Fore.BLUE}Rango de Datos: {START_DATE_STR} a {END_DATE_DISPLAY_STR}{Style.RESET_ALL}")
            print(f"{Fore.BLUE}Fuente de Datos: {data_source}{Style.RESET_ALL}")
            print(f"{Fore.WHITE}{'-'*58}{Style.RESET_ALL}")
            print(f"\n{Fore.CYAN}Métricas de Rendimiento:{Style.RESET_ALL}")
            print(f"  {Fore.CYAN}Capital Inicial: ${metrics['initial_capital']:.2f}{Style.RESET_ALL}")
            print(f"  {Fore.CYAN}Capital Final:   ${metrics['final_capital']:.2f}{Style.RESET_ALL}")
            return_color = Fore.GREEN if metrics['total_return_percent'] >= 0 else Fore.RED
            print(f"  {Fore.CYAN}Retorno Total:   {return_color}{metrics['total_return_percent']:.2f}%{Style.RESET_ALL}")
            print(f"  {Fore.CYAN}Operaciones Totales: {metrics['total_trades']}{Style.RESET_ALL}")
            print(f"  {Fore.CYAN}Operaciones Ganadoras: {metrics['winning_trades']}{Style.RESET_ALL}")
            print(f"  {Fore.CYAN}Operaciones Perdedoras: {metrics['losing_trades']}{Style.RESET_ALL}")
            win_rate_color = Fore.GREEN if metrics['win_rate_percent'] >= 50 else Fore.YELLOW
            print(f"  {Fore.CYAN}Tasa de Ganancias: {win_rate_color}{metrics['win_rate_percent']:.2f}%{Style.RESET_ALL}")
            drawdown_color = Fore.RED if metrics['max_drawdown_percent'] < -10 else Fore.YELLOW if metrics['max_drawdown_percent'] < 0 else Fore.WHITE
            print(f"  {Fore.CYAN}Drawdown Máximo: {drawdown_color}{metrics['max_drawdown_percent']:.2f}%{Style.RESET_ALL}")
            if not np.isnan(metrics['sharpe_ratio']):
                sharpe_color = Fore.GREEN if metrics['sharpe_ratio'] >= 1 else Fore.YELLOW if metrics['sharpe_ratio'] >= 0 else Fore.RED
                print(f"  {Fore.CYAN}Sharpe Ratio:    {sharpe_color}{metrics['sharpe_ratio']:.2f}{Style.RESET_ALL}")
            else:
                print(f"  {Fore.CYAN}Sharpe Ratio:    N/A (no hay suficientes datos para calcular){Style.RESET_ALL}")
            print(f"{Fore.WHITE}{'-'*58}{Style.RESET_ALL}")

            print(f"\n{Fore.CYAN}Registro Detallado de Operaciones:{Style.RESET_ALL}")
            if not trade_log_df.empty:
                log_display_df = trade_log_df[['date', 'type', 'price', 'shares', 'profit_loss']].copy()
                log_display_df.rename(columns={
                    'date': 'Fecha', 'type': 'Tipo', 'price': 'Precio', 'shares': 'Acciones', 'profit_loss': 'Ganancia/Pérdida'
                }, inplace=True)
                log_display_df['Precio'] = log_display_df['Precio'].apply(lambda x: f"${x:.2f}")
                log_display_df['Acciones'] = log_display_df['Acciones'].apply(lambda x: f"{x:.4f}")
                log_display_df['Ganancia/Pérdida'] = log_display_df['Ganancia/Pérdida'].apply(lambda x: f"${x:.2f}" if pd.notna(x) else '-')
                print(tabulate(log_display_df, headers='keys', tablefmt='fancy_grid')) # Usar tabulate
            else:
                print(f"{Fore.YELLOW}No se realizaron operaciones durante el backtest.{Style.RESET_ALL}")
            print(f"{Fore.WHITE}{'='*58}{Style.RESET_ALL}")

            print(f"\n{Fore.CYAN}🔍 Análisis Detallado de Rendimiento:{Style.RESET_ALL}")
            if not trade_log_df.empty and metrics['total_trades'] > 0:
                total_profit = trade_log_df[trade_log_df['profit_loss'] > 0]['profit_loss'].sum()
                total_loss = trade_log_df[trade_log_df['profit_loss'] < 0]['profit_loss'].sum()
                avg_winning_trade = trade_log_df[trade_log_df['profit_loss'] > 0]['profit_loss'].mean()
                avg_losing_trade = trade_log_df[trade_log_df['profit_loss'] < 0]['profit_loss'].mean()
                largest_winning_trade = trade_log_df[trade_log_df['profit_loss'] > 0]['profit_loss'].max()
                largest_losing_trade = trade_log_df[trade_log_df['profit_loss'] < 0]['profit_loss'].min()

                print(f"  {Fore.CYAN}📈 Ganancia Bruta Total: ${total_profit:.2f}{Style.RESET_ALL}")
                print(f"  {Fore.CYAN}📉 Pérdida Bruta Total: ${abs(total_loss):.2f}{Style.RESET_ALL}")
                print(f"  {Fore.CYAN}💲 Ganancia Promedio por Operación Ganadora: ${avg_winning_trade:.2f}{Style.RESET_ALL}")
                print(f"  {Fore.CYAN}🔻 Pérdida Promedio por Operación Perdedora: ${avg_losing_trade:.2f}{Style.RESET_ALL}")
                print(f"  {Fore.CYAN}🏆 Operación Más Grande Ganadora: ${largest_winning_trade:.2f}{Style.RESET_ALL}")
                print(f"  {Fore.CYAN}💀 Operación Más Grande Perdedora: ${largest_losing_trade:.2f}{Style.RESET_ALL}")
                
                print(f"\n{Fore.CYAN}Interpretación del Retorno Final:{Style.RESET_ALL}")
                if metrics['total_return_percent'] >= 0:
                    print(f"  {Fore.GREEN}El retorno positivo de {metrics['total_return_percent']:.2f}% indica que la estrategia logró generar una ganancia neta. Esto ocurre porque el total de las ganancias de las operaciones ganadoras superó al total de las pérdidas de las operaciones perdedoras, o bien, no hubo pérdidas significativas.{Style.RESET_ALL}")
                    if metrics['winning_trades'] > metrics['losing_trades']:
                        print(f"  {Fore.GREEN}La alta tasa de ganancias ({metrics['win_rate_percent']:.2f}%) contribuyó a este resultado, donde la mayoría de las operaciones cerraron con beneficios. Los niveles de Take-Profit (para asegurar ganancias) y Stop-Loss (para limitar pérdidas) ayudaron a gestionar el riesgo.{Style.RESET_ALL}")
                    elif metrics['losing_trades'] > 0:
                         print(f"  {Fore.YELLOW}A pesar de una tasa de ganancias de {metrics['win_rate_percent']:.2f}%, las operaciones ganadoras promedio fueron lo suficientemente grandes como para compensar las pérdidas, llevando a un resultado positivo.{Style.RESET_ALL}")
                else:
                    print(f"  {Fore.RED}El retorno negativo de {metrics['total_return_percent']:.2f}% significa que la estrategia tuvo una pérdida neta. Esto sucede cuando el total de las pérdidas de las operaciones perdedoras superó al total de las ganancias de las operaciones ganadoras.{Style.RESET_ALL}")
                    if metrics['losing_trades'] > metrics['winning_trades']:
                        print(f"  {Fore.RED}La tasa de ganancias de solo {metrics['win_rate_percent']:.2f}% sugiere que más operaciones resultaron en pérdidas que en ganancias. Los niveles de Stop-Loss buscaron limitar estas pérdidas, pero el balance general fue desfavorable.{Style.RESET_ALL}")
                    elif metrics['winning_trades'] > 0:
                         print(f"  {Fore.RED}Aunque hubo operaciones ganadoras, las pérdidas promedio fueron mayores que las ganancias promedio, lo que llevó a un resultado negativo general.{Style.RESET_ALL}")

                print(f"\n{Fore.CYAN}Sobre el Drawdown Máximo:{Style.RESET_ALL}")
                print(f"  {Fore.CYAN}El Drawdown Máximo de {metrics['max_drawdown_percent']:.2f}% representa la mayor caída porcentual desde un pico de capital hasta un valle antes de alcanzar un nuevo pico. Es una medida clave del riesgo y la volatilidad que el portafolio experimentó. Un drawdown alto indica que hubo períodos significativos donde el valor del portafolio se redujo considerablemente desde su punto más alto.{Style.RESET_ALL}")
                print(f"  {Fore.CYAN}Gestionar el drawdown es crucial; el Stop-Loss está diseñado precisamente para limitar estas caídas.{Style.RESET_ALL}")
                
                print(f"\n{Fore.CYAN}Sobre el Sharpe Ratio (Rendimiento Ajustado al Riesgo):{Style.RESET_ALL}")
                if not np.isnan(metrics['sharpe_ratio']):
                    print(f"  {Fore.CYAN}El Sharpe Ratio de {metrics['sharpe_ratio']:.2f} mide el exceso de retorno de tu estrategia por unidad de riesgo asumido (volatilidad). Un Sharpe Ratio más alto generalmente es deseable. Si es > 1, se considera bueno; si es > 2, muy bueno; si es > 3, excelente. Un valor negativo indica que tu estrategia rindió menos que la tasa libre de riesgo, o que tuvo un retorno negativo con alta volatilidad.{Style.RESET_ALL}")
                else:
                    print(f"  {Fore.YELLOW}El Sharpe Ratio no pudo calcularse debido a que no hubo operaciones o la volatilidad de los retornos fue cero (lo que suele ocurrir cuando no hay operaciones).{Style.RESET_ALL}")
            else:
                print(f"{Fore.YELLOW}  No se realizaron operaciones en este backtest, por lo tanto, no hay ganancias o pérdidas que analizar. El capital se mantuvo en el valor inicial.{Style.RESET_ALL}")
            print(f"\n{Fore.WHITE}{'='*58}{Style.RESET_ALL}\n")
            
            plot_backtest_results(historical_data_with_signals, MY_TICKER, INITIAL_CAPITAL, 
                                  strategy_config['name'], strategy_config['std_period'], 
                                  strategy_config['trend_ema_period'], strategy_config['rsi_period'],
                                  strategy_config['macd_periods'],
                                  strategy_config['avwap_anchor_date'],
                                  strategy_config['vwap_crossover_periods'])
        else:
            print(f"{Fore.RED}El backtest para {strategy_config['name']} no pudo completarse. Revisa los mensajes anteriores para más detalles.{Style.RESET_ALL}")
    
    # --- COMPARACIÓN FINAL DE ESTRATEGIAS ---
    if all_strategy_results:
        print(f"\n\n{Fore.CYAN}{'#'*70}{Style.RESET_ALL}")
        print(f"{Fore.CYAN}{' ':10}📊 ANÁLISIS COMPARATIVO FINAL DE ESTRATEGIAS PARA {MY_TICKER} 📊{Style.RESET_ALL}")
        print(f"{Fore.CYAN}{'#'*70}{Style.RESET_ALL}\n")

        comparison_metrics = pd.DataFrame([res['metrics'] for res in all_strategy_results])
        comparison_metrics.set_index('strategy_name', inplace=True)
        
        comparison_metrics_sorted = comparison_metrics.sort_values(by='total_return_percent', ascending=False)

        print(f"{Fore.CYAN}📈 Resumen de Métricas de Todas las Estrategias:{Style.RESET_ALL}")
        print(tabulate(comparison_metrics_sorted[['total_return_percent', 'sharpe_ratio', 'win_rate_percent', 'max_drawdown_percent', 'total_trades']], headers='keys', tablefmt='fancy_grid'))
        print(f"\n{Fore.WHITE}{'-'*70}{Style.RESET_ALL}\n")

        best_strategy = None
        if not comparison_metrics_sorted.empty:
            if 'sharpe_ratio' in comparison_metrics_sorted.columns and comparison_metrics_sorted['sharpe_ratio'].notna().any():
                best_strategy = comparison_metrics_sorted.sort_values(by='sharpe_ratio', ascending=False).iloc[0]
                print(f"{Fore.GREEN}🏆 La ESTRATEGIA CON MEJOR RENDIMIENTO AJUSTADO AL RIESGO (Sharpe Ratio) es: {best_strategy.name}{Style.RESET_ALL}")
                print(f"   {Fore.GREEN}-> Retorno Total: {best_strategy['total_return_percent']:.2f}% | Sharpe Ratio: {best_strategy['sharpe_ratio']:.2f}{Style.RESET_ALL}")
                print(f"   {Fore.GREEN}-> Detalles: Esta estrategia logró el mayor retorno por unidad de riesgo durante el período analizado.{Style.RESET_ALL}")
            else:
                best_strategy = comparison_metrics_sorted.iloc[0]
                print(f"{Fore.GREEN}🏆 La ESTRATEGIA CON MAYOR RETORNO TOTAL es: {best_strategy.name}{Style.RESET_ALL}")
                print(f"   {Fore.GREEN}-> Retorno Total: {best_strategy['total_return_percent']:.2f}%{Style.RESET_ALL}")
                print(f"   {Fore.GREEN}-> Detalles: Esta estrategia generó la mayor ganancia bruta, pero considera que el riesgo no fue ajustado en esta comparación directa.{Style.RESET_ALL}")

            total_profits_series = comparison_metrics['final_capital'] - comparison_metrics['initial_capital']
            winning_strategies = total_profits_series[total_profits_series > 0].index.tolist()
            losing_strategies = total_profits_series[total_profits_series < 0].index.tolist()
            neutral_strategies = total_profits_series[total_profits_series == 0].index.tolist()

            print(f"\n{Fore.CYAN}💰 Análisis de Ganancias y Pérdidas Comparativo:{Style.RESET_ALL}")
            if winning_strategies:
                print(f"  {Fore.GREEN}Estrategias que generaron ganancias: {', '.join(winning_strategies)}{Style.RESET_ALL}")
            if losing_strategies:
                print(f"  {Fore.RED}Estrategias que generaron pérdidas: {', '.join(losing_strategies)}{Style.RESET_ALL}")
            if neutral_strategies:
                print(f"  {Fore.YELLOW}Estrategias sin cambios de capital (no operaron o fueron neutrales): {', '.join(neutral_strategies)}{Style.RESET_ALL}")

            print(f"\n{Fore.WHITE}{'-'*70}{Style.RESET_ALL}\n")
            print(f"{Fore.CYAN}Gráfico Comparativo de Evolución del Portafolio:{Style.RESET_ALL}")
            plot_comparison_results(all_strategy_results, MY_TICKER, INITIAL_CAPITAL)

            print(f"\n{Fore.WHITE}{'-'*70}{Style.RESET_ALL}\n")
            print(f"{Fore.CYAN}Gráfico de Distribución de Ganancias/Pérdidas por Operación:{Style.RESET_ALL}")
            plot_profit_loss_distribution(all_trade_logs_for_comparison, all_strategy_names_for_comparison)

            print(f"\n{Fore.WHITE}{'-'*70}{Style.RESET_ALL}\n")
            export_choice = input(f"{Fore.CYAN}¿Deseas exportar las métricas de comparación y los registros de operaciones a archivos CSV? (s/n): {Style.RESET_ALL}").lower()
            if export_choice == 's':
                export_results_to_csv(all_strategy_results, comparison_metrics_sorted, MY_TICKER)
            else:
                print(f"{Fore.YELLOW}No se exportaron los resultados.{Style.RESET_ALL}")

        else:
            print(f"{Fore.YELLOW}No hay estrategias con resultados válidos para comparar.{Style.RESET_ALL}")

    else:
        print(f"\n{Fore.RED}No se pudieron ejecutar backtests exitosos para ninguna estrategia. Revisa los mensajes anteriores.{Style.RESET_ALL}")
100

👋 Bienvenido/a al Bot de Trading VWAP Avanzado y Amigable!
